In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

#configuration
spark = SparkSession\
    .builder\
    .master('local[2]')\
    .appName('quake_etl')\
    .config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.12:2.4.1')\
    .getOrCreate()  
#load data
vac = spark.read.csv(r"C:\Users\israt\OneDrive\Desktop\DM_Project\project\vaccination.csv", header=True)


#delete column
delete_columns = ['ISO3', 'DATA_SOURCE','DATE_UPDATED','TOTAL_VACCINATIONS_PER100', 'VACCINES_USED','FIRST_VACCINE_DATE',
    'NUMBER_VACCINES_TYPES_USED','PERSONS_BOOSTER_ADD_DOSE_PER100','PERSONS_VACCINATED_1PLUS_DOSE_PER100','PERSONS_FULLY_VACCINATED_PER100']

vac = vac.drop(*delete_columns)
vac.show(1)


#displaying variables types
vac.printSchema()


#convert string to double
vac = vac.withColumn('TOTAL_VACCINATIONS',vac['TOTAL_VACCINATIONS'].cast(DoubleType()))\
    .withColumn('PERSONS_VACCINATED_1PLUS_DOSE',vac['PERSONS_VACCINATED_1PLUS_DOSE'].cast(DoubleType()))\
.withColumn('PERSONS_FULLY_VACCINATED',vac['PERSONS_FULLY_VACCINATED'].cast(DoubleType()))\
.withColumn('PERSONS_BOOSTER_ADD_DOSE',vac['PERSONS_BOOSTER_ADD_DOSE'].cast(DoubleType()))
vac.printSchema()



#count all total
total_t = vac.groupBy('WHO_REGION').sum('TOTAL_VACCINATIONS').withColumnRenamed('sum(TOTAL_VACCINATIONS)', 'TOTAL VACCINATION')
total_f = vac.groupBy('WHO_REGION').sum('PERSONS_VACCINATED_1PLUS_DOSE').withColumnRenamed('sum(PERSONS_VACCINATED_1PLUS_DOSE)', 'TOTAL PERSONS VACCINATED 1PLUS DOSE')
total_s = vac.groupBy('WHO_REGION').sum('PERSONS_FULLY_VACCINATED').withColumnRenamed('sum(PERSONS_FULLY_VACCINATED)', 'TOTAL FULLY VACCINATED')
total_b = vac.groupBy('WHO_REGION').sum('PERSONS_BOOSTER_ADD_DOSE').withColumnRenamed('sum(PERSONS_BOOSTER_ADD_DOSE)', 'TOTAL BOOSTER DOSE GIVEN')
total_t.show(20)
total_f.show(20)
total_s.show(20)
total_b.show(20)



# connect with mongodb by creating database
vac.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/vaccination.vac').save()\

total_t.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/vaccination.TOTAL_VACCINATION').save()\

total_f.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/vaccination.TOTAL PERSONS VACCINATED 1PLUS DOSE').save()\

total_s.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/vaccination.TOTAL FULLY VACCINATED').save()\

total_b.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/vaccination.TOTAL BOOSTER DOSE GIVEN').save()


+-----------+----------+------------------+-----------------------------+------------------------+------------------------+
|    COUNTRY|WHO_REGION|TOTAL_VACCINATIONS|PERSONS_VACCINATED_1PLUS_DOSE|PERSONS_FULLY_VACCINATED|PERSONS_BOOSTER_ADD_DOSE|
+-----------+----------+------------------+-----------------------------+------------------------+------------------------+
|Afghanistan|      EMRO|          11391703|                     10615478|                 9771963|                    null|
+-----------+----------+------------------+-----------------------------+------------------------+------------------------+
only showing top 1 row

root
 |-- COUNTRY: string (nullable = true)
 |-- WHO_REGION: string (nullable = true)
 |-- TOTAL_VACCINATIONS: string (nullable = true)
 |-- PERSONS_VACCINATED_1PLUS_DOSE: string (nullable = true)
 |-- PERSONS_FULLY_VACCINATED: string (nullable = true)
 |-- PERSONS_BOOSTER_ADD_DOSE: string (nullable = true)

root
 |-- COUNTRY: string (nullable = true)
 |